In [40]:
import pandas as pd 
import numpy as np 
from __future__ import division

# Inputs vairables

Below is an example of a two door (four channels total), low floor bus, with smart card fare payment system. 

This bus has five boardings, two alightings, no standees while departure from a bus stop. 

We assume this bus lost three seconds while openning and closing bus doors. 

In [2]:
fare_payment_method = 'Smart card'
percent_using_farebox = 0.45 #45% is our default, feel free to change it

In [42]:
num_doors = 2
boarding_doors = 2

In [43]:
num_channels = 4
#channel = 'front_door'
channel_boards = 1 # boarding channel number 
channel_exits = 3 # alighting channel number

In [49]:
tot_boards = 5
tot_exits = 2
standees = 0

In [50]:
boarding_height = 'stairs'

In [52]:
vehicle = {}
vehicle["door open close time"] = 3

# Boarding / alingting via channels

In [53]:
def calculate_boards_by_channel(num_channels, boarding_doors, percent_using_farebox):
    
    pct_boards_by_channel = { 1: 0.00, 2: 0.00, 3: 0.00, 4: 0.00, 5: 0.00, 6: 0.00 } 

    ##-- Situation A: single-channel boarding w/ all fares paid or inspected upon boarding
    if num_channels == 1:
        pct_boards_by_channel[1] = 1.00
        return pct_boards_by_channel
    
    ##-- Situation B: If need to have fare inspected or paid, assume double-channel boarding. 
    ##  Channel 1 is for people who need to interact with farebox.  
    ##  Channel 2 is for those who just need a visual inspection.

    if fare_payment_method != 'none':
        pct_boards_by_channel[1] = percent_using_farebox
        pct_boards_by_channel[2] = 1.00 - percent_using_farebox
        return pct_boards_by_channel
  
    ##-- Situation C: All door boarding, free or pre-paid fares

    # Based on Exhibit 6-58 and TCRP 299, a function of available door channels
    # NOTE: Derived from Exhibit 4-3 in TCQSM, 2nd Edition (5).
    # It can be assumed that boarding passengers are evenly divided among the remaining door channels. 

    PCT_BOARDS_USING_BUSIEST_CHANNEL = { 
        1: 1.00, 
        2: 0.6, 
        3: 0.45, 
        4: 0.35, 
        5: 0.3, 
        6: 0.25,
        }
    if boarding_doors == 'front':
        pct_boards_by_channel[1] = percent_using_farebox
        pct_boards_by_channel[2] = 1 - pct_boards_by_channel[1]
    else:
        pct_boards_by_channel[1] = PCT_BOARDS_USING_BUSIEST_CHANNEL[num_channels]
        # spread remaining percentage evenly out among remaining channels
        for ch in range(2, num_channels+1):
            pct_boards_by_channel[ch] = (1 - pct_boards_by_channel[1])/(num_channels-1)
            
    return pct_boards_by_channel

In [54]:
def calculate_exits_by_channel(num_channels, boarding_doors, percent_using_farebox):
    pct_exits_by_channel = { 1: 0.00, 2: 0.00, 3: 0.00, 4: 0.00, 5: 0.00, 6: 0.00 } 

    ##-- Situation A: single-channel 
    if num_channels == 1:
        pct_exits_by_channel[1] = 1.00
        return pct_exits_by_channel
    
    ##-- Situation B: single door
    if num_doors == 1:
        pct_exits_by_channel[2] = 1.00
        return pct_exits_by_channel
        
    ##-- Situation C: two doors with all boardings thru front, assume 25% use front door and 75% rear door, split between channels
    if num_doors == 2 and boarding_doors == "front":
        pct_exits_by_channel[2] = 0.25
        pct_exits_by_channel[3] = 0.75 / (num_channels - 2)
        if num_channels >= 4: pct_exits_by_channel[4] = 0.75 / (num_channels - 2)
        return pct_exits_by_channel
    
    ##-- Situation D: two or more doors or all-door boarding
    
    PCT_EXITS_USING_BUSIEST_CHANNEL = { 
        2: 0.75, 
        3: 0.45, 
        4: 0.35, 
        5: 0.30, 
        6: 0.25,
        } 
        
    pct_exits_by_channel[3] = PCT_EXITS_USING_BUSIEST_CHANNEL[num_channels]
    pct_exits_by_channel[2] = ( 1.0 - PCT_EXITS_USING_BUSIEST_CHANNEL[num_channels] ) / (num_channels - 2)
    for ch in [4, 5, 6]:
        if num_channels >= ch: 
            pct_exits_by_channel[ch] = ( 1.0 - PCT_EXITS_USING_BUSIEST_CHANNEL[num_channels] ) / (num_channels - 2)
    
    return pct_exits_by_channel

# Boarding / alightings service time via channels

In [55]:
def calculate_boarding_service_time(fare_payment_method, standees, boarding_height, channel_boards, channel_exits):

    ## SOURCE: TCQSM 3rd edition Exhibit 6-4
    # Seconds per passenger
    AVG_BOARD_TIME_BY_FARE_METHOD = {
        'None'                : 1.75, 
        'Visual inspection'   : 2.00, 
        'Single ticket/token' : 3.00, 
        'Exact change'        : 4.50,
        'Ticket validator'    : 4.00, 
        'Magstripe card'      : 5.00, 
        'Smart card'          : 2.75,
        }
    
    avg_board_time = AVG_BOARD_TIME_BY_FARE_METHOD[fare_payment_method]
    
    ## NOTE: Add 0.5 s/p to boarding times when standees are present. 
    ##       Add 0.5 s/p for non-level boarding (1.0 s/p for motor coaches).     
    if standees:                            avg_board_time += 0.5
    if boarding_height   == "stairs":       avg_board_time += 0.5
    elif boarding_height == "steep stairs": avg_board_time += 1.0
    
    ## When more than 25% of the passenger flow through a single door channel is in the 
    ## opposite direction of the main flow of passengers, increase both boarding and 
    ## alighting service times by 20% to account for passenger congestion at the door
    pct_boards = (channel_boards*1.0)/(1.0*(channel_boards + channel_exits))
    pct_exits  = 1-pct_boards
    if min(pct_boards, pct_exits) > 0.25:
        avg_board_time = avg_board_time * 1.20
    
    channel_board_time = channel_boards * avg_board_time
    return channel_board_time


In [56]:
def calculate_exit_service_time(channel, channel_boards, channel_exits):

    ## SOURCE: TCQSM 3rd edition Exhibit 6-4
    AVG_EXIT_TIME_BY_DOOR = {
    'front_door'    : 2.50,
    'rear_door'     : 1.75,
    }
    if channel >= 2: 
        channel = 'rear_door'
    else: 
        channel = 'front_door'
    
    avg_exit_time = AVG_EXIT_TIME_BY_DOOR[channel]
    
    ## When more than 25% of the passenger flow through a single door channel is in the 
    ## opposite direction of the main flow of passengers, increase both boarding and 
    ## alighting service times by 20% to account for passenger congestion at the door
    pct_boards = (1.0*channel_boards)/(1.0*(channel_boards + channel_exits))
    pct_exits  = 1 - pct_boards
    if min(pct_boards, pct_exits) > 0.25:
        avg_exit_time = avg_exit_time * 1.20
        
    channel_exit_time = channel_exits * avg_exit_time
    return channel_exit_time

# Maximum flow time via channels

In [69]:
def get_channel_flow_time(channel, tot_boards, tot_exits, pct_boards_by_channel, pct_exits_by_channel, standees, boarding_height, fare_payment_method):
    print '########channel##########: ', channel
    
    #calc channel boards, exits
    channel_boards = tot_boards*pct_boards_by_channel[channel]
    print 'channel boarding:', channel_boards
    
    print tot_exits
    channel_exits  = tot_exits*pct_exits_by_channel[channel]
    print 'channel alighting:', channel_exits
    
    channel_board_time = calculate_boarding_service_time(fare_payment_method, standees, boarding_height, channel_boards, channel_exits)
    channel_exit_time  = calculate_exit_service_time(channel, channel_boards, channel_exits)
    print 'channel boarding time:', channel_board_time
    print 'channel exit time:', channel_exit_time
    return channel_board_time + channel_exit_time #note that it doesn't make sense to me that this is necessarily additive



Variables from GTFS
- "percent_using_farebox"
- "fare_payment_method"
- "boarding_height" [ level, stairs, steep stairs ]
- "door open close time" (2-5 sec) [2,3,4,5]
- "boarding doors" [front, all]
- "num_channels"

Variables from FT
- "standees present" [True, False]
- "tot_boards"
- "tot_exits"


In [70]:
pct_boards_by_channel = calculate_boards_by_channel(num_channels, boarding_doors, percent_using_farebox)

In [71]:
pct_exits_by_channel  = calculate_exits_by_channel(num_channels, boarding_doors, percent_using_farebox)

In [72]:
channel_flow_times = [get_channel_flow_time(channel, 
                                            tot_boards, 
                                            tot_exits, 
                                            pct_boards_by_channel, 
                                            pct_exits_by_channel, 
                                            standees, boarding_height, 
                                            fare_payment_method) 
                      for channel in range(1, num_channels+1)] 

########channel##########:  1
channel boarding: 2.25
2
channel alighting: 0.0
channel boarding time: 7.3125
channel exit time: 0.0
########channel##########:  2
channel boarding: 2.75
2
channel alighting: 0.65
channel boarding time: 8.9375
channel exit time: 1.1375
########channel##########:  3
channel boarding: 0.0
2
channel alighting: 0.7
channel boarding time: 0.0
channel exit time: 1.225
########channel##########:  4
channel boarding: 0.0
2
channel alighting: 0.65
channel boarding time: 0.0
channel exit time: 1.1375


In [73]:
channel_flow_times

[7.3125, 10.075, 1.2249999999999999, 1.1375]

In [62]:
#channel_flow_times = [get_channel_flow_time(channel, tot_boards, tot_exits, pct_boards_by_channel, pct_exits_by_channel, standees, boarding_height, fare_payment_method) for ch in num_channels]

In [63]:
pax_flow_time = max(channel_flow_times)

# Boarding lost time

In [64]:
# Used to calculate efficiency loss from having multiple boarding areas.  Range is from 0 - 8 seconds for 1 - 5 boarding areas.
# For FT, assume single boarding area, which results in zero lost time.
boarding_lost_time = 0

In [65]:
# get value from vehicle_ft, else use Fast-Trips default of 2.5 [just threw that out there ]
# validate to be between 2-5 seconds

door_open_close_time = vehicle["door open close time"]

In [66]:
total_dwell = boarding_lost_time + door_open_close_time + pax_flow_time

In [67]:
print total_dwell

13.075


In [68]:
print 'done with one bus-routes & stop!'

done with one bus-routes & stop!
